In [1]:
# Importing required libraries
import math
import numpy as np
from numpy.random import choice

In [2]:
# Defining the gates
x = [[0, 1], [1, 0]]
h = [[0.70710678, 0.70710678], [0.70710678, -0.70710678]]

# Initialised cx as zeros because it'll later be calculated using projection and x operator
cx = [[0, 0], [0, 0]]

# Define projection operator |0><0|
P0x0 = np.array([
[1, 0],
[0, 0]
])

# Define projection operator |1><1|
P1x1 = np.array([
[0, 0],
[0, 1]
])

In [3]:
# Function to define the ground state of 'num_qubits' number of qubits
def get_ground_state(num_qubits):
    v = []
    for i in range(2**num_qubits):
        if i == 0:
            v.append(1)
        else:
            v.append(0)
    return v

# Function to calculate the unitary operator of size 2**n x 2**n for given gate and target qubits
def get_operator(total_qubits, gate_unitary, target_qubits):
    
    # Code for single qubit gates
    if gate_unitary == x or gate_unitary == h:
        
        # Identity I
        I = np.identity(2)
        
        # Flag variable keeps track of position of the gate U
        flag = 0
    
        # Initialize operator O  as I x U or U x I if target qubit is in first or second position
        if target_qubits[0] == 0:
            O = np.kron(gate_unitary, I)
            flag = 1
        elif target_qubits[0] == 1:
            O = np.kron(I, gate_unitary)
            flag = 1
    
        # Calculate the final operator O based on various cases
        for i in range((total_qubits - 1) - flag):
            if flag == 1:
                O = np.kron(O, I)
            elif i == (target_qubits[0] - 1):
                O = np.kron(O, gate_unitary)
            elif i == 0:
                O = np.kron(I, I)
            else:
                O = np.kron(O, I)      
    
    # Code for controlled-x
    elif gate_unitary == cx:
        
        # Identity I
        I = np.identity(2)
        
        # Since cx is flipping target qubit based on control qubit, we initialize gate_unitary as x
        gate_unitary = x
        
        # flag1 keeps track of projection operator/control qubit
        flag1 = 0
        
        # flag2 keeps track of target qubit
        flag2 = 0
    
        # Code if control qubit is in first position
        if target_qubits[0] == 0:
            if target_qubits[1] == 1:
                first_term = np.kron(P0x0, I)
                second_term = np.kron(P1x1, gate_unitary)
                flag2 = 1
            else:
                first_term = np.kron(P0x0, I)
                second_term = np.kron(P1x1, I)
            flag1 = 1
        
        # Code if control qubit is in second position
        elif target_qubits[0] == 1:
            if target_qubits[1] == 0:
                first_term = np.kron(I, P0x0)
                second_term = np.kron(gate_unitary, P1x1)
                flag2 = 1
            else:
                first_term = np.kron(I, P0x0)
                second_term = np.kron(I, P1x1)
            flag1 = 1
    
        # Calculate the first and second terms of O simultaneously based on various cases
        for i in range((total_qubits - 1) - flag1):
            if flag1 == 1 and flag2 == 1:
                first_term = np.kron(first_term, I)
                second_term = np.kron(second_term, I)
            
            elif flag1 == 0 and i == (target_qubits[0] - 1):
                first_term = np.kron(first_term, P0x0)
                second_term = np.kron(second_term, P1x1)
                flag1 = 1
            
            elif flag2 == 0 and i == (target_qubits[1] - 2):
                first_term = np.kron(first_term, I)
                second_term = np.kron(second_term, gate_unitary)
                flag2 = 1        
        
            elif flag1 == 0 and flag2 == 0 and i == 0:
                first_term = np.kron(I, I)
                second_term = np.kron(I, I)
            
            else:
                first_term = np.kron(first_term, I)
                second_term = np.kron(second_term, I)
    
        # Adding the two terms simultaneously calculated to get O
        O = first_term + second_term
    
    return O

# Function to run the circuit
def run_program(initial_state, program):

    final_state = initial_state
    
    # Calculating operator O and multiplying state with O for each gate
    for i in range(len(program)):
        O = get_operator(int(math.log(len(my_qpu))/math.log(2)), program[i]["gate"], program[i]["target"])
        final_state = np.dot(final_state, O)
    return final_state

# Function to measure qubits 'num_shots' number of times
def measure_all(state_vector, num_shots):
    
    # Stores the binary indexes of n number of qubits
    index = []
    
    # Calculating the binary representation and storing in index
    for i in range(len(state_vector)):
        index.append(np.binary_repr(i, width = int(math.log(len(my_qpu))/math.log(2))))

    # Measuring the qubits using weighted random
    measurements = choice(index, num_shots, p = np.abs(state_vector)**2)

    return measurements

# Function to calculate the counts of results
def get_counts(state_vector, num_shots):
    
    results = measure_all(state_vector, num_shots)
    unique, counts = np.unique(results, return_counts=True)
    counts = dict(zip(unique, counts))
    
    return counts

In [4]:
# Create "quantum computer" with 2 qubits (code works for any number of qubits)
my_qpu = get_ground_state(2)

In [5]:
# Defining the circuit
my_circuit = [
{ "gate": h, "target": [0] },
{ "gate": cx, "target": [0, 1]}
]

In [6]:
# Run circuit
final_state = run_program(my_qpu, my_circuit)
print("Initial State: ",my_qpu)
print("Final State: ",final_state)

Initial State:  [1, 0, 0, 0]
Final State:  [0.70710678 0.         0.         0.70710678]


In [8]:
# Getting results after measuring
results = get_counts(final_state, 500)

print(results)

{'00': 249, '11': 251}
